In [1]:
import json
from collections import namedtuple, OrderedDict

class Universe(object):

    FILENAME = 'universe-pretty.json'
    
    Node = namedtuple('Node', 'x y z'.split())
    
    def __init__(self):
        _universe = json.load(open(self.FILENAME, 'r'))
        self._solar_systems = OrderedDict()
        for i in _universe['solarSystems']:
            self._solar_systems[i['id']] = i
        self._jumps = _universe['jumps']
        
        self._nodes = OrderedDict()
        for i_id, i_solar_system in self._solar_systems.items():
            self._nodes[i_id] = self.Node(x=i_solar_system['x'], y=i_solar_system['y'], z=i_solar_system['z'])
    
    def nodes_coords(self, coord):
        return [i[coord] for i in self._solar_systems.values()]

    def edges_coords(self, coord):
        result = []
        for i_jump in self._jumps:
            from_node = self._nodes[i_jump['from']]
            to_node = self._nodes[i_jump['to']]
            result += [getattr(from_node, coord), getattr(to_node, coord), None]
        return result

In [14]:
import networkx as nx

universe = Universe()

g = nx.Graph()
g.add_nodes_from(universe._nodes.keys())
g.add_edges_from([(i['from'], i['to']) for i in universe._jumps])

In [15]:
import eveil

# Sheroo
# Cistuvaert
# Ethernity
# Vay
# Azer
# Clellinon
# Jita

# r = nx.shortest_simple_paths(
#     g,
#     eveil.System('Sosh').system_id,
#     eveil.System('Sheroo').system_id,
# )
# list(r)

r = nx.shortest_simple_paths(
    g,
    30000003,
    30000001,
)
list(r)

KeyboardInterrupt: 

In [5]:
# Create a graph with all solar-systems in the Universe.
universe = Universe()

print('JUMPS COUNT:', len(universe._jumps))

regions = set()
regions_connections = set()
for i_jump in universe._jumps:
    from_system = universe._solar_systems[i_jump['from']]
    to_system = universe._solar_systems[i_jump['to']]
    from_region = from_system['region']
    to_region = to_system['region']
    if from_region != to_region:
        regions.add(from_region)
        regions.add(to_region)
        regions_connections.add(tuple(sorted([from_region, to_region])))

# for i_from, i_to in sorted(regions_connections):
#     print('CONNECTED REGIONS:', i_from, 'TO', i_to)

import networkx as nx
regions_graph = nx.Graph()
regions_graph.add_nodes_from(regions)
for i_region_connection in regions_connections:
    regions_graph.add_edge(*i_region_connection)

# nx.write_yaml(regions_graph, 'regions.yaml')
    
# nearby = nx.shortest_path_length(regions_graph, 'The Forge')
# nearby = [(j, i) for i,j in nearby.items()]
# nearby.sort()
# nearby

JUMPS COUNT: 13826


In [18]:
from plotly.graph_objs import Scatter3d, Line, Data, Figure, Layout, Marker
from plotly import offline

offline.init_notebook_mode(connected=True)

universe = Universe()

# Nodesd
nodes_data = Scatter3d(
    x=universe.nodes_coords('x'),
    y=universe.nodes_coords('y'),
    z=universe.nodes_coords('z'),
    mode='markers',
    marker=Marker(symbol='dot', color='#3333FF', size=3),
    hoverinfo='none',
)
edges_data = Scatter3d(
    x=universe.edges_coords('x'),
    y=universe.edges_coords('y'),
    z=universe.edges_coords('z'),
    mode='lines',
    line=Line(color='#F11', width=1),
    hoverinfo='none',
)
fig = Figure(
    data=Data([nodes_data, edges_data]),
    layout=Layout(
        title='Universe'
    )
)
offline.iplot(fig)